In [10]:
import os
from dotenv import load_dotenv, find_dotenv

from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.tools import Tool
from langchain.agents import initialize_agent
import requests
from bs4 import BeautifulSoup


In [7]:
dotenv_file = '/home/amir/source/.env'
load_dotenv(dotenv_file)

True

In [ ]:
api_key = os.getenv("OPENAI_API_KEY")
print(api_key)

In [11]:
prompt_template = """Write a LinkedIn post about {topic}, following these guidelines:

- Make the post concise and appealing
- Use emojis when suitable
"""

llm = OpenAI(temperature=0.7)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

print(llm_chain('Elon Musk')['text']) 

/tmp/ipykernel_450120/2328495582.py:7: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.7)
/tmp/ipykernel_450120/2328495582.py:8: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  llm_chain = LLMChain(
/tmp/ipykernel_450120/2328495582.py:13: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  print(llm_chain('Elon Musk')['text'])



🚀 Exciting news! 🤩 Elon Musk, the visionary behind SpaceX, Tesla, and Neuralink, has just been named the world's richest person! 💰 His innovative ideas and relentless drive have truly paid off. 🙌 Let's all congratulate Elon and continue to be inspired by his incredible achievements. 👏 #ElonMusk #Innovation #Visionary #Inspiration #SpaceX #Tesla #Neuralink #RichestPerson #Congratulations 🎉


In [49]:
def get_article(_):
    url = 'https://techcrunch.com/2023/10/02/elon-musk-is-being-sued-for-defamation-after-falsely-claiming-a-man-was-a-neo-nazi-on-x/'
    # content = requests.get(url).text
    # article = BeautifulSoup(content).find_all('div', {'class': 'article-content'})[0]
    # return '\n\n'.join([p.text for p in article.find_all('p', recursive=False)])
    
    response = requests.get(url)
    response.raise_for_status()  # Ensure the request was successful
    
    # Parse the HTML
    soup = BeautifulSoup(response.text, "html.parser")

    # Extract the text
    if soup:
        article_text = soup.get_text(separator="\n", strip=True)
        # print(article_text)
    else:
        print("Main article content not found.")
        
    return article_text
    

In [50]:
elon_musk_retriever = Tool.from_function(
    func=get_article,
    name='Elon Musk Article Retriever',
    description='Useful when you want to retrieve an article about Elon Musk'
)

In [51]:
prompt_template = """Write a LinkedIn post based on the following article. Use these guidelines:
- Make the post concise and appealing
- Use emojis when suitable

{article}
"""

llm = OpenAI(temperature=0.7)
llm_chain = LLMChain(
    llm=llm,
    prompt=PromptTemplate.from_template(prompt_template)
)

linkedin_post_writer = Tool.from_function(
    func=lambda article: llm_chain(article)['text'],
    name='LinkedIn Post Writer',
    description='Useful when you want to write a LinkedIn post based on the given article.'
)

In [52]:
agent = initialize_agent(tools=[elon_musk_retriever, linkedin_post_writer],
                         llm=OpenAI(temperature=0),
                         verbose=True)

agent.run('Please write a LinkedIn post about Elon Musk sued')



> Entering new AgentExecutor chain...
 I should research the latest news about Elon Musk and the lawsuit
Action: Elon Musk Article Retriever
Action Input: "Elon Musk sued"
Observation: Elon Musk is being sued for defamation after falsely claiming a man was a neo-Nazi on X | TechCrunch
Elon Musk is being sued for defamation after falsely claiming a man was a neo-Nazi on X | TechCrunch
TechCrunch Desktop Logo
TechCrunch Mobile Logo
Latest
Startups
Venture
Apple
Security
AI
Apps
Events
Podcasts
Newsletters
Sign In
Search
Submit
Site Search Toggle
Mega Menu Toggle
Topics
Latest
AI
Amazon
Apps
Biotech & Health
Climate
Cloud Computing
Commerce
Crypto
Enterprise
EVs
Fintech
Fundraising
Gadgets
Gaming
Google
Government & Policy
Hardware
Instagram
Layoffs
Media & Entertainment
Meta
Microsoft
Privacy
Robotics
Security
Social
Space
Startups
TikTok
Transportation
Venture
More from TechCrunch
Events
Startup Battlefield
StrictlyVC
Newsletters
Podcasts
Videos
Partner Content
TechCrunch Brand Studio

'Elon Musk is being sued for defamation after falsely claiming a man was a neo-Nazi on X.'

In [47]:
get_article(_)

Elon Musk is being sued for defamation after falsely claiming a man was a neo-Nazi on X | TechCrunch
Elon Musk is being sued for defamation after falsely claiming a man was a neo-Nazi on X | TechCrunch
TechCrunch Desktop Logo
TechCrunch Mobile Logo
Latest
Startups
Venture
Apple
Security
AI
Apps
Events
Podcasts
Newsletters
Sign In
Search
Submit
Site Search Toggle
Mega Menu Toggle
Topics
Latest
AI
Amazon
Apps
Biotech & Health
Climate
Cloud Computing
Commerce
Crypto
Enterprise
EVs
Fintech
Fundraising
Gadgets
Gaming
Google
Government & Policy
Hardware
Instagram
Layoffs
Media & Entertainment
Meta
Microsoft
Privacy
Robotics
Security
Social
Space
Startups
TikTok
Transportation
Venture
More from TechCrunch
Events
Startup Battlefield
StrictlyVC
Newsletters
Podcasts
Videos
Partner Content
TechCrunch Brand Studio
Crunchboard
Contact Us
Sign In
Image Credits:
Nathan Laine/Bloomberg / Getty Images
Social
Elon Musk is being sued for defamation after falsely claiming a man was a neo-Nazi on X
Taylor 

"Elon Musk is being sued for defamation after falsely claiming a man was a neo-Nazi on X | TechCrunch\nElon Musk is being sued for defamation after falsely claiming a man was a neo-Nazi on X | TechCrunch\nTechCrunch Desktop Logo\nTechCrunch Mobile Logo\nLatest\nStartups\nVenture\nApple\nSecurity\nAI\nApps\nEvents\nPodcasts\nNewsletters\nSign In\nSearch\nSubmit\nSite Search Toggle\nMega Menu Toggle\nTopics\nLatest\nAI\nAmazon\nApps\nBiotech & Health\nClimate\nCloud Computing\nCommerce\nCrypto\nEnterprise\nEVs\nFintech\nFundraising\nGadgets\nGaming\nGoogle\nGovernment & Policy\nHardware\nInstagram\nLayoffs\nMedia & Entertainment\nMeta\nMicrosoft\nPrivacy\nRobotics\nSecurity\nSocial\nSpace\nStartups\nTikTok\nTransportation\nVenture\nMore from TechCrunch\nEvents\nStartup Battlefield\nStrictlyVC\nNewsletters\nPodcasts\nVideos\nPartner Content\nTechCrunch Brand Studio\nCrunchboard\nContact Us\nSign In\nImage Credits:\nNathan Laine/Bloomberg / Getty Images\nSocial\nElon Musk is being sued for

In [18]:
url = 'https://techcrunch.com/2023/10/02/elon-musk-is-being-sued-for-defamation-after-falsely-claiming-a-man-was-a-neo-nazi-on-x/'
content = requests.get(url).text

In [20]:
BeautifulSoup(content).find_all('div', {'class': 'article-content'})

[]

In [21]:
response = requests.get(url)
response.raise_for_status()  # Ensure the request was successful

# Parse the HTML
soup = BeautifulSoup(response.text, "html.parser")

# Extract the main article content
article_div = soup.find("div", class_="article-content")

In [38]:
article_div

In [41]:
soup.get_text()

# Extract the text
if soup:
    article_text = soup.get_text(separator="\n", strip=True)
    print(article_text)
else:
    print("Main article content not found.")

Elon Musk is being sued for defamation after falsely claiming a man was a neo-Nazi on X | TechCrunch
Elon Musk is being sued for defamation after falsely claiming a man was a neo-Nazi on X | TechCrunch
TechCrunch Desktop Logo
TechCrunch Mobile Logo
Latest
Startups
Venture
Apple
Security
AI
Apps
Events
Podcasts
Newsletters
Sign In
Search
Submit
Site Search Toggle
Mega Menu Toggle
Topics
Latest
AI
Amazon
Apps
Biotech & Health
Climate
Cloud Computing
Commerce
Crypto
Enterprise
EVs
Fintech
Fundraising
Gadgets
Gaming
Google
Government & Policy
Hardware
Instagram
Layoffs
Media & Entertainment
Meta
Microsoft
Privacy
Robotics
Security
Social
Space
Startups
TikTok
Transportation
Venture
More from TechCrunch
Events
Startup Battlefield
StrictlyVC
Newsletters
Podcasts
Videos
Partner Content
TechCrunch Brand Studio
Crunchboard
Contact Us
Sign In
Image Credits:
Nathan Laine/Bloomberg / Getty Images
Social
Elon Musk is being sued for defamation after falsely claiming a man was a neo-Nazi on X
Taylor 

In [40]:
soup.get_text(separator="\n", strip=True)

"Elon Musk is being sued for defamation after falsely claiming a man was a neo-Nazi on X | TechCrunch\nElon Musk is being sued for defamation after falsely claiming a man was a neo-Nazi on X | TechCrunch\nTechCrunch Desktop Logo\nTechCrunch Mobile Logo\nLatest\nStartups\nVenture\nApple\nSecurity\nAI\nApps\nEvents\nPodcasts\nNewsletters\nSign In\nSearch\nSubmit\nSite Search Toggle\nMega Menu Toggle\nTopics\nLatest\nAI\nAmazon\nApps\nBiotech & Health\nClimate\nCloud Computing\nCommerce\nCrypto\nEnterprise\nEVs\nFintech\nFundraising\nGadgets\nGaming\nGoogle\nGovernment & Policy\nHardware\nInstagram\nLayoffs\nMedia & Entertainment\nMeta\nMicrosoft\nPrivacy\nRobotics\nSecurity\nSocial\nSpace\nStartups\nTikTok\nTransportation\nVenture\nMore from TechCrunch\nEvents\nStartup Battlefield\nStrictlyVC\nNewsletters\nPodcasts\nVideos\nPartner Content\nTechCrunch Brand Studio\nCrunchboard\nContact Us\nSign In\nImage Credits:\nNathan Laine/Bloomberg / Getty Images\nSocial\nElon Musk is being sued for